# OAT for c-LCA

## This is a Jupyter notebook to run sensitivity, uncertainty, analysis in LCA using parameters.

Author: Dineshkumar Muniyappan
Date: 6 February 2026

This notebook introduces the notion behind parameters and provides dedicated sections for: One-at-a-Time (OAT)

Acknowledgements: 
This notebook builds based on previous work developed within the ALIGNED project, and workshops of Rebecca Belfoire, and Christhel Andrade. Thanks Lorie Hamelin to provide time to learn this analysis. 


In [ ]:
# -------------------------------
# Import bw2.5 core packages
# -------------------------------

import bw2calc as bc 
# bw2calc: performs Life Cycle Impact Assessment (LCIA) calculations
# - LCA object (bc.LCA)
# - lci(), lcia()
# Used every time you re-run the LCA during OAT
import bw2data as bd
# bw2data: manages Brightway data structures
# - projects, databases, activities, exchanges, parameters
# - setting current project
# - reading/modifying parameters and exchanges
import bw2io as bi
# bw2io: input/output utilities
# - importing Excel / CSV LCIs
# - database importers
# Used when loading or updating LCI datasets
import bw2analyzer as bwa
# bw2analyzer: post-processing and contribution analysis
# - contribution analysis
# - supply chain traversal
# Optional but useful for interpreting OAT results
import bw_processing as bwp
# bw_processing: efficient array storage for LCA matrices
# - used internally by Brightway
# - required for Monte Carlo and GSA workflows
# Usually not called directly, but must be imported

# -------------------------------
# Brightway parameter system
# -------------------------------
from bw2data.parameters import parameters, ActivityParameter, DatabaseParameter
# parameters: central registry of all Brightway parameters
# ActivityParameter: parameters attached to activities
# DatabaseParameter: parameters attached to databases
# These are essential for parametric LCA and OAT
from bw2data.backends.schema import ExchangeDataset
# ExchangeDataset: low-level representation of an exchange
# Used to directly modify exchange amounts during OAT
# (only needed if parameters are NOT used everywhere)
from lci_to_bw2 import *
# Your script that:
# - defines how LCI is converted into Brightway databases
# - may contain functions to build or update the model
from Importer_parameters import *
# Your script that:
# - reads Excel parameter files
# - creates or updates Brightway parameters
# - links Excel values to BW parameters
from Harmonizer import *
# Your script that:
# - harmonizes datasets (units, naming, structure)
# - ensures consistency across databases
# Important before running OAT to avoid hidden inconsistencies

# -------------------------------
# Scientific Python stack
# -------------------------------
import pandas as pd
# pandas: data handling
# - reading Excel files
# - storing OAT results
# - building result tables
import matplotlib.pyplot as plt
# matplotlib: plotting library
# - line plots
# - tornado plots for OAT results
import seaborn as sns
# seaborn: statistical plotting on top of matplotlib
# - cleaner plots
# - useful for sensitivity visualizations
from scipy import stats
# scipy.stats: statistical functions
# - distributions
# - correlation coefficients
# - sensitivity metrics (if needed)
import numpy as np
# numpy: numerical computing
# - arrays
# - mathematical operations
# - used everywhere in parametric analysis

In [ ]:
bd.projects
#Brightway project manager

In [ ]:
bd.projects.set_current('pyro_phenol') #  name of your project 
bd.databases

In [ ]:
import os

bi.import_ecoinvent_release(
    version='3.11',
    system_model='consequential',
    username = os.getenv('ECOINVENT_USER'),   
    password= os.getenv('ECOINVENT_PASS')  

)
# Importing the ecoinvent and biosphere database this might take around 10-15min.

if (os.getenv('ECOINVENT_USER') is None) or (os.getenv('ECOINVENT_PASS') is None):
    raise ValueError("Set ECOINVENT_USER and ECOINVENT_PASS environment variables before running the ecoinvent import.")


# -------------------------------
# Importing the foreground datbases
# -------------------------------

In [ ]:
#import marginal supplem databases
ei = bi.ExcelImporter('1. Marginal biomass.xlsx') # to import the database in the project
ei.apply_strategies()
ei.match_database('ecoinvent-3.11-biosphere') # to match the project to biosphere
ei.match_database('ecoinvent-3.11-consequential') # to match the project to ecoinvent

ei.statistics() # Get the overview of our LCI in Brightway. Check if there are unlinked exchanges

# to activate parameters
ei.write_project_parameters() 
ei.write_database(activate_parameters=True)

In [ ]:
#import marginal supplem databases
ei = bi.ExcelImporter('2. Marginal_suppliers_20250519.xlsx') # to import the database in the project
ei.apply_strategies()
ei.match_database('ecoinvent-3.11-biosphere') # to match the project to biosphere
ei.match_database('ecoinvent-3.11-consequential') # to match the project to ecoinvent

ei.statistics() # Get the overview of our LCI in Brightway. Check if there are unlinked exchanges

# to activate parameters
ei.write_project_parameters() 
ei.write_database(activate_parameters=True)

In [ ]:
bd.databases

In [ ]:
#Load Foreground database
ei = bi.ExcelImporter('3. Pyro Phenol_v14_CHP.xlsx') # to import the database in the project

ei.apply_strategies()
ei.match_database('ecoinvent-3.11-biosphere') # to match the project to biosphere
ei.match_database('ecoinvent-3.11-consequential') # to match the project to ecoinvent
ei.match_database('Marginal biomass') # to match the marginal softwood production
ei.match_database('marginal_suppliers_20250519') # to match the marginal energy
#ei.match_database('Background processes') # to match the CHP, factory, etc. in Pyro_phenol

ei.statistics() 

# to activate parameters
ei.write_project_parameters() 
ei.write_database(activate_parameters=True)

In [ ]:
bd.databases

In [ ]:
# Link parameters to activities 
for a in bd.Database('Pyro Phenol CHP'):  #bd.Database('Pyro Phenol CHP') refers to a Brightway database. This loop iterates over every activity (process) inside the 'Pyro Phenol CHP' database
    for e in a.exchanges():    #Every activity in Brightway can have exchanges (inputs and outputs). An exchange represents the flow of materials/energy between activities. e represents one exchange for the current activity a.
        dict_ex = dict(e)   # This converts the exchange e into a dictionary (dict_ex), which is a more flexible data structure for manipulation.
        if e.get('group') is not None:   #This checks if the exchange e has a group associated with it. group is a custom field used for grouping exchanges, often used to categorize them (e.g.,"pyro_group_CHP" for a specific category of exchanges).
            parameters.add_exchanges_to_group(e['group'],a)  #parameters.add_exchanges_to_group is a method that links an exchange to a parameter group. e['group'] is the group of the current exchange (e.g., pyro_group_CHP).
            break

In [ ]:
##CHECK THAT ALL FORMULAS ARE CORRECTLY WRITTEN

# Pick the database
db = bd.Database("Pyro Phenol CHP")

# Build a dictionary of all parameter values
param_context = {p.name: p.amount for p in ActivityParameter}

# Container for results
exchange_checks = []

for act in db:
    activity_name = act.get("name", "")
    activity_code = act["code"]
    for exc in act.exchanges():
        formula = exc.get("formula", None)
        stored_amount = exc["amount"]
        if formula:
            try:
                # Evaluate the formula in python using the parameter values
                calculated = eval(formula, {}, param_context)
                diff = calculated - stored_amount
            except Exception as e:
                calculated = f"ERROR: {e}"
                diff = None

            exchange_checks.append({
                "Activity name": activity_name,
                "Activity code": activity_code,
                "Exchange input": str(exc.input),
                "Stored amount": stored_amount,
                "Formula": formula,
                "Calculated from formula": calculated,
                "Difference": diff
            })

# Put into a DataFrame for inspection
df_checks = pd.DataFrame(exchange_checks)

# Show a few entries
print("Sample of formula checks:")
print(df_checks.head(10))

# Optionally, filter for large differences
tol = 1e-6  # tolerance
df_large_diff = df_checks[df_checks["Difference"].apply(lambda x: (x is not None) and abs(x) > tol)]
print("\nExchanges with significant mismatch:")
print(df_large_diff.head(10))

# If needed, write to Excel
df_checks.to_excel("exchange_formula_validation_CHP_2.xlsx", index=False)
print("\nResults saved to exchange_formula_validation_CHP_2.xlsx")

In [ ]:
#Find activities in the database --> find the code of FU to use it later
for a in bd.Database('Pyro Phenol CHP'):
    print(a, a['code'])

In [ ]:
# pick one activity from the ones you have listed 
activity = bd.Database('Pyro Phenol CHP').get('5999fc8533d24713977b500f12598eab')
activity

In [ ]:
# let us look at the feature of the all the exchanges of our actvity
for e in activity.exchanges():
    for feature in e:
        print(feature, ':', e[feature])
    print()

In [ ]:
# check that all parameters are loaded
for p in ActivityParameter:
    print(p.name,p.group, p.amount)

In [ ]:
sorted(bd.methods)   #It represents a collection (such as a list, tuple, or other iterable) that contains various methods. These methods could represent different Life Cycle Assessment (LCA) methodologies, databases, or other functions.

In [ ]:
# Define impact categories
ac_method = ('ecoinvent-3.11', 'EF v3.1', 'acidification', 'accumulated exceedance (AE)') #acidification
cc_method = ('ecoinvent-3.11', 'IPCC 2021 (incl. biogenic CO2)', 'climate change: total (incl. biogenic CO2)', 'global warming potential (GWP100)') #climate change  IPCC
ecoF_method = ('ecoinvent-3.11', 'EF v3.1', 'ecotoxicity: freshwater', 'comparative toxic unit for ecosystems (CTUe)') #ecotoxicity freshwater 
ecoFi_method = ('ecoinvent-3.11', 'EF v3.1', 'ecotoxicity: freshwater, inorganics', 'comparative toxic unit for ecosystems (CTUe)') #ecotoxicity freshwater inorganics
ecoFo_method = ('ecoinvent-3.11', 'EF v3.1', 'ecotoxicity: freshwater, organics', 'comparative toxic unit for ecosystems (CTUe)') #ecotoxicity freshwater organics
pm_method = ('ecoinvent-3.11', 'EF v3.1', 'particulate matter formation', 'impact on human health') #particulate matter
euF_method = ('ecoinvent-3.11', 'EF v3.1', 'eutrophication: freshwater', 'fraction of nutrients reaching freshwater end compartment (P)') #eutrophication freshwater P
euM_method = ('ecoinvent-3.11', 'EF v3.1', 'eutrophication: marine', 'fraction of nutrients reaching marine end compartment (N)') #eutrophication marine N
euT_method = ('ecoinvent-3.11', 'EF v3.1', 'eutrophication: terrestrial', 'accumulated exceedance (AE)') #eutrophication terrestrial
huTox_method = ('ecoinvent-3.11', 'EF v3.1', 'human toxicity: carcinogenic', 'comparative toxic unit for human (CTUh)') #Human toxicity cancer
huToxi_method = ('ecoinvent-3.11', 'EF v3.1', 'human toxicity: carcinogenic, inorganics', 'comparative toxic unit for human (CTUh)') #Human toxicity cancer inorganics
huToxo_method = ('ecoinvent-3.11', 'EF v3.1', 'human toxicity: carcinogenic, organics', 'comparative toxic unit for human (CTUh)') #Human toxicity cancer organics
huToxNoCancer_method = ('ecoinvent-3.11', 'EF v3.1', 'human toxicity: non-carcinogenic', 'comparative toxic unit for human (CTUh)') #Human toxicity non cancer
huToxNoCanceri_method = ('ecoinvent-3.11', 'EF v3.1', 'human toxicity: non-carcinogenic, inorganics', 'comparative toxic unit for human (CTUh)') #Human toxicity non cancer inorganics
huToxNoCancero_method = ('ecoinvent-3.11', 'EF v3.1', 'human toxicity: non-carcinogenic, organics', 'comparative toxic unit for human (CTUh)') #Human toxicity non cancer organics
ir_method = ('ecoinvent-3.11', 'EF v3.1', 'ionising radiation: human health', 'human exposure efficiency relative to u235') #ionising radiation
lu_method = ('ecoinvent-3.11', 'EF v3.1', 'land use', 'soil quality index') #land use
od_method = ('ecoinvent-3.11', 'EF v3.1', 'ozone depletion', 'ozone depletion potential (ODP)') #ozone depletion
pho_method = ('ecoinvent-3.11', 'EF v3.1', 'photochemical oxidant formation: human health', 'tropospheric ozone concentration increase') #Photochemical ozone formation
rf_method = ('ecoinvent-3.11', 'EF v3.1', 'energy resources: non-renewable', 'abiotic depletion potential (ADP): fossil fuels') #resource fossils
rm_method = ('ecoinvent-3.11', 'EF v3.1', 'material resources: metals/minerals', 'abiotic depletion potential (ADP): elements (ultimate reserves)') #resource materials minerals
wu_method = ('ecoinvent-3.11', 'EF v3.1', 'water use', 'user deprivation potential (deprivation-weighted water consumption)') #water use"

methods = [ac_method, cc_method, ecoF_method, ecoFi_method, ecoFo_method, pm_method, euF_method, euM_method, euT_method, huTox_method, huToxi_method, huToxo_method, huToxNoCancer_method, huToxNoCanceri_method, 
           huToxNoCancero_method, ir_method, lu_method, od_method, pho_method, rf_method, rm_method, wu_method]

method = [cc_method, pm_method, euF_method, euM_method, rm_method, pm_method, wu_method]

In [ ]:
#define FU --> Here FU is one activity containing all the process
FU_CHP = bd.Database('Pyro Phenol CHP').get('5999fc8533d24713977b500f12598eab')   #FU for CHP scenario
FU_CHP

In [ ]:
# save databases exact names for easy use
biosphere = 'ecoinvent-3.11-biosphere'
ecoinvent = 'ecoinvent-3.11-consequential'
marginal = 'marginal_suppliers_20250519'
biomass = 'Marginal biomass'

Pyro_phenol = 'Pyro Phenol CHP'

In [ ]:
#This is to define which parameters to assign to the corresponding db when using more than 1 db. This Python code snippet is used to filter and assign specific parameters to a corresponding database (db)

CHP_parameters = []  #This initializes an empty list called CHP_parameters that will hold the parameters that belong to the group pyro_group_CHP.
groups = ['pyro_group_CHP']  #This is a list containing the group name(s) that will be used for filtering. In this case, the group to look for is 'pyro_group_CHP'.

for p in ActivityParameter:  #This is a loop that iterates over each p in ActivityParameter. ActivityParameter likely refers to a collection or list of parameter objects (or attributes), such as those in an LCA framework (e.g., Brightway2).
    if p.group in groups:  #This condition checks if the group attribute of each parameter (p.group) is in the groups list. If a parameter's group is 'pyro_group_CHP' (or any other group in groups), the condition is True.
        CHP_parameters.append(p) #If the condition is met, the parameter p is added to the CHP_parameters list. This means only parameters belonging to the 'pyro_group_CHP' group will be included.
   

In [ ]:
originals_VALUES = []   #This initializes an empty list called originals_VALUES that will store the amount values extracted from each ActivityParameter.
for p in ActivityParameter:  #This is a for loop that iterates over each object p in the ActivityParameter collection. This suggests that ActivityParameter is likely a list or iterable of parameter objects in your context (e.g., from an LCA model, database, or analysis framework).
    originals_VALUES.append(p.amount) #For each ActivityParameter object p, the code appends the amount attribute of that object (p.amount) to the originals_VALUES list.

In [ ]:
#This Python code snippet iterates over each element in the ActivityParameter collection and calls the recalculate_exchanges method on each object (p). Specifically, it passes the group attribute of the object to the method.
for p in ActivityParameter:  #This is a for loop that iterates through each ActivityParameter object in the ActivityParameter collection. ActivityParameter is likely a list or iterable containing multiple parameter objects related to activities in an LCA framework or model.
    p.recalculate_exchanges(p.group) #For each ActivityParameter object p, the recalculate_exchanges method is called. p.group is passed as an argument to this method. group is likely an attribute of p that refers to a specific group or category the parameter belongs to (e.g., a group for a specific LCA method or a process type like 'pyro_group_CHP').

In [ ]:
#Scans every Brightway2 database in your current project:Scans every activity in each database:Scans every exchange (technosphere + biosphere + production, etc.) in each activity:For each exchange, it reads the exchange "amount":It skips exchanges where "amount" is missing (None).For each bad exchange, it stores a record: It prints How many bad exchanges were found
import numpy as np  #Imports the NumPy library, which is typically used for numerical operations, such as checking if a value is a scalar or performing array operations.
import bw2data as bd #Imports the Brightway2 data module (bw2data) and aliases it as bd. Brightway2 is an open-source framework for Life Cycle Assessment (LCA) that allows you to work with environmental data.

bad_exchanges = [] #Initializes an empty list called bad_exchanges, which will be used to store exchanges that have non-scalar amount values.

for db_name in bd.databases: #Loops through all databases available in the Brightway2 project (bd.databases), which is an iterable containing the names of the databases.
    db = bd.Database(db_name)  #Loads each database by its name using bd.Database(db_name). This creates a Database object, allowing access to its data.
    for act in db: #Loops through each activity in the current database (db). An activity in an LCA context typically represents a process or stage in the system (e.g., manufacturing, transportation, waste disposal).
        for exc in act.exchanges(): #Loops through all the exchanges associated with the current activity (act). Exchanges in LCA represent inputs and outputs for a given activity, such as raw materials, emissions, or energy use.
            amt = exc.get("amount", None) #Attempts to retrieve the value of the amount attribute for each exchange (exc). If the amount attribute is not present, None is returned.
            if amt is None: #If the amount is None (i.e., the exchange doesn’t have a value for amount), it skips that exchange and continues to the next one.
                continue
            if not np.isscalar(amt): #This checks whether the amount is not a scalar. np.isscalar() returns True if the value is a scalar (i.e., a single number). If amt is not a scalar (e.g., it's a list or array), the code proceeds to the next line.
                bad_exchanges.append((db_name, act.key, exc.input, amt)) #If the exchange has a non-scalar amount, it appends a tuple containing:

print("❌ Exchanges with non-scalar amounts:", len(bad_exchanges))  #After processing all databases and exchanges, this prints the total number of exchanges that have non-scalar amounts.
for e in bad_exchanges[:10]: #Loops through the first 10 "bad exchanges" in the bad_exchanges list (if any) and prints them to give the user insight into what exchanges are problematic.
    print(e) #Prints each "bad exchange", showing the database name, activity key, input, and the non-scalar amount.


In [ ]:
#This Python code scans a specific database in Brightway2 (in this case, "Pyro Phenol CHP") and checks all exchanges for activities within that database. It looks for exchanges that have non-numeric amount values, and if any are found, it appends them to the bad list.
bad = []

for act in bd.Database("Pyro Phenol CHP"):
    for exc in act.exchanges():
        amt = exc.get("amount", None)
        if amt is None:
            continue

        # If amount is not a pure number
        if not isinstance(amt, (int, float)):
            bad.append((act.key, exc.input.key, amt, type(amt)))

print(f"Found {len(bad)} non‑numeric amounts")
for a in bad:
    print(a)


## Static LCA ##

In [ ]:
# calculate
#This Python code snippet calculates the LCA (Life Cycle Assessment) score for different methods using the Brightway2 framework. It stores the calculated scores and prints the results for each method. Let's go through the code line-by-line:
static_scores = []  #Initializes an empty list called static_scores, which will be used to store the LCA scores for each method.
for m in methods:
    lca = bc.LCA(demand={FU_CHP:1}, method = m)
    lca.lci()
    lca.lcia()
    static_scores.append(lca.score)
    print(f"The {m[3]} is {lca.score} {bd.methods[m]['unit']}")

In [ ]:
# Define the Functional Unit (FU) and FU_conv
FU = bd.Database('Pyro Phenol CHP').get('5999fc8533d24713977b500f12598eab')  # Original FU
FU_conv = bd.Database('Pyro Phenol CHP').get('5999fc8533d24713977b500f12598eab')  # Define FU_conv for conversion analysis (replace with the correct key)

# Impact methods
methods = [
    ('ecoinvent-3.11', 'EF v3.1', 'climate change', 'global warming potential (GWP100)'),
    ('ecoinvent-3.11', 'EF v3.1', 'eutrophication: freshwater', 'fraction of nutrients reaching freshwater end compartment (P)'),
    ('ecoinvent-3.11', 'EF v3.1', 'eutrophication: marine', 'fraction of nutrients reaching marine end compartment (N)'),
    ('ecoinvent-3.11', 'EF v3.1', 'material resources: metals/minerals', 'abiotic depletion potential (ADP): elements (ultimate reserves)'),
    ('ecoinvent-3.11', 'EF v3.1', 'particulate matter formation', 'impact on human health'),
    ('ecoinvent-3.11', 'EF v3.1', 'water use', 'user deprivation potential (deprivation-weighted water consumption)'),
]

# Excel export
output_path = "contribution_analysis_full_export_conventional.xlsx"
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    for method in methods:
        print(f"\n🔍 Method: {method[3]}")
        try:
            # LCA
            lca = bc.LCA({FU_conv: 1}, method=method)
            lca.lci()
            lca.lcia()
            total_score = lca.score

            # Contribution analysis
            ca = bwa.ContributionAnalysis()
            contributions = ca.annotated_top_processes(lca, limit=None)

            # Export all to DataFrame
            
            df = pd.DataFrame(contributions, columns=["Share of Impact", "Activity Key", "Process"])
            df["% of Total"] = df["Share of Impact"] * 100
            df["% of Total"] = df["Share of Impact"] / total_score * 100
            df["Total Score"] = total_score
            df.to_excel(writer, sheet_name=method[3][:31], index=False)

            print(f"  ✔️ Exported {len(df)} contributors to Excel.")

        except Exception as e:
            print(f"  ❌ Error in {method[3]}: {e}")

print(f"\n✅ All done. Saved to: {output_path}")


Calculate LCA for each activity in the system to build the contribution analysis. Here, we iterate inside FU to retrieve the name and amount of each activity and then perform LCA per activity and then compile all activities together to get contribution per activity

In [ ]:
# Define impact methods
methods = [
    ('ecoinvent-3.11', 'EF v3.1', 'climate change', 'global warming potential (GWP100)'),
    ('ecoinvent-3.11', 'EF v3.1', 'eutrophication: freshwater', 'fraction of nutrients reaching freshwater end compartment (P)'),
    ('ecoinvent-3.11', 'EF v3.1', 'eutrophication: marine', 'fraction of nutrients reaching marine end compartment (N)'),
    ('ecoinvent-3.11', 'EF v3.1', 'material resources: metals/minerals', 'abiotic depletion potential (ADP): elements (ultimate reserves)'),
    ('ecoinvent-3.11', 'EF v3.1', 'particulate matter formation', 'impact on human health'),
    ('ecoinvent-3.11', 'EF v3.1', 'water use', 'user deprivation potential (deprivation-weighted water consumption)')
]

# Your database name


# Get the FU activity
FU_conv = bd.Database('Pyro Phenol CHP').get('5999fc8533d24713977b500f12598eab')

# Extract technosphere exchanges from FU
exchanges = list(FU_conv.technosphere())
results = []

for exc in exchanges:
    try:
        input_act = bd.get_activity(exc.input.key)
        amount = exc.amount
        row = {
            "Activity": input_act.get("name", str(input_act.key)),
            "Code": input_act.key[1],
            "Used amount in FU": amount
        }

        for method in methods:
            lca = bc.LCA({input_act: amount}, method=method)
            lca.lci()
            lca.lcia()
            row[method[3]] = lca.score

        results.append(row)

    except Exception as e:
        print(f"Error processing exchange from {exc.input.key}: {e}")

# Save to Excel in working directory
df = pd.DataFrame(results)
df.to_excel("lca_each_activity_from_FU_conventional.xlsx", index=False)
print("✅ Saved: lca_each_activity_from_FU_conventional.xlsx")


In [ ]:
##This Python code performs Life Cycle Assessment (LCA) on the technosphere exchanges of a given functional unit (FU) from a Brightway2 database (FU_conv) using multiple impact methods. It then exports the results, including contribution analysis, to an Excel file for each combination of exchange and method.

# Set your database name
#Hdr_db = 'Hedgerow_LCI2'
Conv_farm = 'Conventional_farm'

# Get the main FU activity
#FU_hr = bd.Database(Hdr_db).get('e0a6324d979640de9883a89ee8d9ce84')
FU_conv = bd.Database('Pyro Phenol CHP').get('5999fc8533d24713977b500f12598eab')



# Define impact methods
methods = [
    ('ecoinvent-3.11', 'EF v3.1', 'climate change', 'global warming potential (GWP100)'),
    ('ecoinvent-3.11', 'EF v3.1', 'particulate matter formation', 'impact on human health'),
    ('ecoinvent-3.11', 'EF v3.1', 'eutrophication: freshwater', 'fraction of nutrients reaching freshwater end compartment (P)'),
    ('ecoinvent-3.11', 'EF v3.1', 'eutrophication: marine', 'fraction of nutrients reaching marine end compartment (N)'),
    ('ecoinvent-3.11', 'EF v3.1', 'material resources: metals/minerals', 'abiotic depletion potential (ADP): elements (ultimate reserves)'),
    ('ecoinvent-3.11', 'EF v3.1', 'water use', 'user deprivation potential (deprivation-weighted water consumption)'),
]

# Extract all technosphere exchanges from FU
exchanges = list(FU_conv.technosphere())

# Excel output file
output_path = "contribution_analysis_multi_fu_conv.xlsx"
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:

    # Loop through each activity used in the main FU
    for exc in exchanges:
        try:
            input_act = bd.get_activity(exc.input.key)
            amount = exc.amount
            act_name = input_act.get('name', str(input_act.key))

            for method in methods:
                method_name = method[3]
                print(f"\n🔍 {act_name} | {method_name}")

                # Run LCA
                lca = bc.LCA({input_act: amount}, method=method)
                lca.lci()
                lca.lcia()
                total_score = lca.score

                # Contribution analysis
                ca = bwa.ContributionAnalysis()
                contributions = ca.annotated_top_processes(lca, limit=None)

                # Build DataFrame
                df = pd.DataFrame(contributions, columns=["Absolute Impact", "Activity Key", "Process"])
                df["% of Total"] = df["Absolute Impact"] / total_score * 100
                df["Total Score"] = total_score
                df["FU Activity"] = act_name
                df["Method"] = method_name

                # Sheet name: truncate if too long
                sheet_name = f"{act_name[:15]} - {method_name[:15]}"
                df.to_excel(writer, sheet_name=sheet_name[:31], index=False)

        except Exception as e:
            print(f"❌ Error with activity {exc.input.key} | {method[3]}: {e}")

print(f"\n✅ All results exported to: {output_path}")


## **OAT** ##

Change "for p in XX_parameter" according to the group paramater database correpsonding to the system modelling, ex. "for p in Conv_parameter" will run parameters in conventional farming LCI while "for p in Hdr_parameter" will run parameters in hedgerows LCI.

In [ ]:
# This function updates a given parameter’s amount, saves the updated value, and then recalculates the exchanges for the parameter based on its group. It ensures that the system reflects the updated parameter in the associated exchanges (e.g., inputs/outputs in an LCA model).
def update_parameter(parameter, amount):
    parameter.amount = amount
    parameter.save()

    parameter.recalculate_exchanges(parameter.group)

In [ ]:
# This code is performing a sensitivity analysis on the parameters of an LCA model using the Brightway2 framework. It calculates how changes in specific parameters affect the overall LCA score for multiple impact methods. Here's a detailed breakdown of each part:
# Container to hold all method results
results = {}

for m in methods:
    print(m[2])    
    # initialise the data
    par_name = []
    par_values = []
    lca_scores = []
    SR_values  = []

    # calculate starting LCA score
    lca = bc.LCA(demand={FU_CHP: 1}, method = m)
    lca.lci()
    lca.lcia()
    start = lca.score

    for p in CHP_parameters:
    #for p in ActivityParameter:  
        original = p.amount  # store the initial value

        par_values.append(original)
        par_name.append(p.name)

        # change the parameter value
        update_parameter(p, original * 1.1)

        # recalculate the LCA score
        lca = bc.LCA(demand={FU_CHP: 1}, method = m)
        lca.lci()
        lca.lcia()
        new_score = lca.score

        # save the result and print to screen
        lca_scores.append(new_score)
        print(f'{p.name}: \t {new_score}')

        # reset the parameter back to its original value
        update_parameter(p, original)

        # OPTIONAL: verify score is back to base after reset
        lca = bc.LCA(demand={FU_CHP: 1}, method = m)
        lca.lci()
        lca.lcia()
        print(f"Reset score check for {p.name}: {lca.score}")

        SR = ((new_score-start)/start) / (0.1)
        SR_values.append(SR)

    results[m[2]] = [par_name, par_values, lca_scores, SR_values]

    print('---'*30)

In [ ]:
# to have all the results and export them in excel
dfs = []

for impact_category in results:
    par_name, par_values, lca_scores, SR_values = results[impact_category]
    df = pd.DataFrame({
        'parameter names': par_name,
        'parameter values': par_values,
        impact_category + '\nresults': lca_scores,
        'Sensitivity ratio': SR_values
    })
    dfs.append(df)

In [ ]:
#exporting the results in excel - separate files per impact
import os
import re

# Make an output folder (optional but recommended)
output_dir = "exported_results_conv_v4" #Make sure to change the naming for each FU
os.makedirs(output_dir, exist_ok=True)

for df in dfs:
    # Use column 2 (impact category) as base for filename
    base_name = df.columns[2]  # e.g. "material resources: metals\nresults"
    
    # Clean up filename: replace illegal characters with underscore
    safe_base = re.sub(r'[<>:"/\\|?*\n]+', '_', base_name)  # replace invalid chars + newlines
    safe_base = re.sub(r'\s+', '_', safe_base).strip('_')   # replace multiple spaces with underscores

    # Build final path
    file_path = os.path.join(output_dir, f"{safe_base}.xlsx")

    # Export to Excel
    df.to_excel(file_path, index=False)
    print(f"Saved: {file_path}")

In [ ]:
sr_df = pd.DataFrame()

for impact_category in results:
    par_name, par_values, lca_scores, SR_values = results[impact_category]
    temp_df = pd.DataFrame({impact_category: SR_values}, index=par_name)
    sr_df = pd.concat([sr_df, temp_df], axis=1)

sr_df.head(20)

In [ ]:
import matplotlib.pyplot as plt
import math

# Step 1: Combine SR results into a DataFrame
sr_df = pd.DataFrame()

for impact_category in results:
    par_name, par_values, lca_scores, SR_values = results[impact_category]
    temp_df = pd.DataFrame({impact_category: SR_values}, index=par_name)
    sr_df = pd.concat([sr_df, temp_df], axis=1)

# Step 2: Remove dummy parameters
sr_df = sr_df[~sr_df.index.str.startswith("__dummy_")]

# Optional: Sort by impact (e.g., climate change)
# sr_df = sr_df.sort_values(by='climate change', ascending=False)

# Step 3: Split into two parts (keeping labels accurate)
half = math.ceil(len(sr_df) / 2)
df_top = sr_df.iloc[:half].copy()
df_bottom = sr_df.iloc[half:].copy()

# Reset index to ensure seaborn shows correct labels
df_top.index.name = "Parameter"
df_bottom.index.name = "Parameter"

# Create the plot
sns.set(style="white")
cmap = sns.diverging_palette(240, 10, as_cmap=True)

fig, axs = plt.subplots(1, 2, figsize=(18, max(8, half * 0.4)), sharey=False)

sns.heatmap(df_top, annot=True, fmt='.3f', cmap=cmap, cbar=True, vmin=-3, vmax=3, ax=axs[0],
            yticklabels=True)

sns.heatmap(df_bottom, annot=True, fmt='.3f', cmap=cmap, cbar=True, vmin=-3, vmax=3, ax=axs[1],
            yticklabels=True)

axs[0].set_title("OAT SR results (Top Half)")
axs[1].set_title("OAT SR results (Bottom Half)")

plt.tight_layout()
plt.savefig("OAT_SR_split_heatmap_CHP.png", dpi=300, bbox_inches="tight")  #make sure of using the right saving name
plt.show()


In [ ]:
print(sr_df.shape)
print(sr_df.head())
print(sr_df.isna().sum().sum())  # number of NaN values
